In [ ]:
import pandas as pd
import numpy as np
import joblib
import logging
import time

In [ ]:
inference_data = pd.read_csv('../data/raw/stores_sales_forecasting_updated_v3.1.csv', 
                 sep=';',
                 encoding='utf-8')
inference_data.drop(['Row ID'], axis =1, inplace=True)
inference_data.head()